In [17]:
# pip install datasets
import pandas as pd
import polars as pl

In [2]:
import pandas as pd
# from datasets import load_dataset
# import polars as pl

In [3]:
df = pd.read_parquet("/home/jupyter/datasphere/project/part-0.parquet")

In [4]:
nan_ratio = df.isna().sum() / len(df)
nan_ratio

inn               3.089372e-07
ogrn              0.000000e+00
region            8.600811e-03
region_taxcode    3.089372e-07
creation_date     0.000000e+00
                      ...     
line_6325         9.960561e-01
line_6326         9.882533e-01
line_6330         9.894164e-01
line_6350         9.758105e-01
line_6400         8.602609e-01
Length: 213, dtype: float64

In [6]:
nan_threshold = 0.25
good_columns = nan_ratio[nan_ratio <= 0.1].index.tolist()
good_columns

['inn',
 'ogrn',
 'region',
 'region_taxcode',
 'creation_date',
 'age',
 'eligible',
 'financial',
 'filed',
 'outlier',
 'okved',
 'okved_section',
 'okopf',
 'okogu',
 'okfc',
 'oktmo',
 'lon',
 'lat',
 'geocoding_quality']

In [7]:
df = df[good_columns]

In [30]:
df.to_csv("/home/jupyter/datasphere/project/from-s3-folder/finilized_first_dataset.csv")

In [1]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import requests

In [4]:
url = "https://data.rcsi.science/data-catalog/datasets/205/#"  # Replace with the actual URL
save_path = "/home/jupyter/datasphere/project/205.zip"  # Replace with your desired filename and path

In [5]:
response = requests.get(url)

In [ ]:
response

In [1]:
zip_file_path = "/home/jupyter/datasphere/project/205.zip"
csv_file_in_zip_name = 'panel.csv'
parquet_output_path = "/home/jupyter/datasphere/project/205.parquet"
csv_file_path = "/home/jupyter/datasphere/project/giant_csv_dataset.csv"

In [3]:
pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import zipfile
import polars as pl
import os

print(f"Начинается процесс обработки ZIP-архива: '{zip_file_path}'")
print(f"Будет извлечен CSV-файл: '{csv_file_in_zip_name}'")
print(f"Итоговый Parquet-файл будет сохранен по пути: '{parquet_output_path}'")

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        if csv_file_in_zip_name not in zip_ref.namelist():
            raise FileNotFoundError(
                f"Файл '{csv_file_in_zip_name}' не найден внутри архива '{zip_file_path}'. "
                f"Доступные файлы: {zip_ref.namelist()}"
            )

        print(f"Чтение '{csv_file_in_zip_name}' напрямую из архива в память...")

        with zip_ref.open(csv_file_in_zip_name) as csv_file_stream:
            with pl.StringCache():
                # Указываем dtypes для колонки 'tin' и null_values для некорректных данных
                df = pl.read_csv(
                    csv_file_stream,
                    has_header=True,
                    infer_schema_length=10000,
                    separator=';',
                    encoding='utf8', # Попробуйте 'cp1251' или 'latin1', если utf8 не работает
                    # --- КЛЮЧЕВЫЕ ИЗМЕНЕНИЯ ЗДЕСЬ ---
                    dtypes={'tin': pl.Utf8}, # Указываем 'tin' как строковый тип данных
                    null_values=["~~~~~~~~~~~~"], # Явно указываем эту строку как значение NULL
                    # ignore_errors=True, # Можно использовать как крайнюю меру, если выше не поможет
                )
    print(f"CSV-файл успешно загружен в Polars DataFrame. Количество строк: {len(df)}")
    print("Проверка заголовков после корректного чтения:")
    print(df.columns)
    print(f"Пример данных из колонки 'tin':")
    print(df['tin'].head())
    print(f"Пример данных из колонки 'region':")
    print(df['region'].head())

except zipfile.BadZipFile:
    print(f"Ошибка: Файл '{zip_file_path}' не является действительным ZIP-архивом или поврежден.")
except FileNotFoundError as e:
    print(f"Ошибка: {e}. Проверьте правильность пути к ZIP-архиву или имени CSV-файла внутри него.")
except pl.ComputeError as e:
    print(f"Произошла ошибка Polars при обработке файла: {e}")
    print("Возможно, проблемы с определением схемы или форматом данных. Попробуйте настроить параметры `pl.read_csv`.")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

Начинается процесс обработки ZIP-архива: '/home/jupyter/datasphere/project/205.zip'
Будет извлечен CSV-файл: 'panel.csv'
Итоговый Parquet-файл будет сохранен по пути: '/home/jupyter/datasphere/project/205.parquet'
Чтение 'panel.csv' напрямую из архива в память...


Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.


CSV-файл успешно загружен в Polars DataFrame. Количество строк: 63855163
Проверка заголовков после корректного чтения:
['tin', 'year', 'reg_number', 'kind', 'category', 'org_name', 'org_short_name', 'activity_code_main', 'region_iso_code', 'region_code', 'region', 'area', 'settlement', 'settlement_type', 'oktmo', 'lat', 'lon', 'start_date', 'end_date', 'revenue', 'expenditure', 'employees_count']
Пример данных из колонки 'tin':
shape: (10,)
Series: 'tin' [str]
[
	"000000000071"
	"000000000071"
	"000000610911"
	"000000610911"
	"002461011626"
	"002461011626"
	"0100000011"
	"0100000011"
	"0100000011"
	"0100000011"
]
Пример данных из колонки 'region':
shape: (10,)
Series: 'region' [str]
[
	"Тульская облас…
	"Тульская облас…
	"Ростовская обл…
	"Ростовская обл…
	"Красноярский к…
	"Красноярский к…
	"Республика Ады…
	"Республика Ады…
	"Республика Ады…
	"Республика Ады…
]


In [5]:
print(df.columns)

['tin', 'year', 'reg_number', 'kind', 'category', 'org_name', 'org_short_name', 'activity_code_main', 'region_iso_code', 'region_code', 'region', 'area', 'settlement', 'settlement_type', 'oktmo', 'lat', 'lon', 'start_date', 'end_date', 'revenue', 'expenditure', 'employees_count']


In [8]:
print(f"Начинается запись в Parquet файл: '{parquet_output_path}'")

df.write_parquet(
    parquet_output_path,
    compression='snappy',
)

print(f"Файл успешно преобразован в Parquet и сохранен по пути: {parquet_output_path}")

Начинается запись в Parquet файл: '/home/jupyter/datasphere/project/205.parquet'
Файл успешно преобразован в Parquet и сохранен по пути: /home/jupyter/datasphere/project/205.parquet


In [9]:
import os

file_path = "/home/jupyter/datasphere/project/205.parquet"  # Replace with the actual path to your file

file_size = os.path.getsize(file_path)
print(f"The size of '{file_path}' is {file_size // 1024 // 1024} megabytes.")

The size of '/home/jupyter/datasphere/project/205.parquet' is 1971 megabytes.


In [10]:
import pandas as pd
df = pd.read_parquet("/home/jupyter/datasphere/project/205.parquet", engine="fastparquet")

In [8]:
# pip install dask

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
df.columns

Index(['tin', 'year', 'reg_number', 'kind', 'category', 'org_name',
       'org_short_name', 'activity_code_main', 'region_iso_code',
       'region_code', 'region', 'area', 'settlement', 'settlement_type',
       'oktmo', 'lat', 'lon', 'start_date', 'end_date', 'revenue',
       'expenditure', 'employees_count'],
      dtype='object')

In [ ]:
df.head(10)['kind']

In [3]:
import zipfile
import os
num_lines_to_read = 2 # Количество строк для просмотра (включая заголовок)

print(f"Попытка прочитать первые {num_lines_to_read} строк из '{csv_file_in_zip_name}' внутри архива '{zip_file_path}'...")

try:
    # Открываем ZIP-архив
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Проверяем, существует ли указанный CSV-файл внутри архива
        if csv_file_in_zip_name not in zip_ref.namelist():
            raise FileNotFoundError(
                f"Файл '{csv_file_in_zip_name}' не найден внутри архива '{zip_file_path}'. "
                f"Доступные файлы: {zip_ref.namelist()}"
            )

        # Открываем CSV-файл как потоковый объект
        with zip_ref.open(csv_file_in_zip_name, 'r') as csv_file_stream:
            # Читаем и выводим указанное количество строк
            for i, line_bytes in enumerate(csv_file_stream):
                if i >= num_lines_to_read:
                    break
                # Декодируем байты в строку. Используйте правильную кодировку!
                # Чаще всего это 'utf-8', но для русскоязычных данных может быть 'cp1251'.
                # Если первая попытка не удалась, попробуйте 'cp1251'.
                try:
                    line = line_bytes.decode('utf-8').strip().split(';')
                except UnicodeDecodeError:
                    line = line_bytes.decode('cp1251', errors='ignore').strip() # 'errors=ignore' для избежания ошибок
                print(line)

except zipfile.BadZipFile:
    print(f"Ошибка: Файл '{zip_file_path}' не является действительным ZIP-архивом или поврежден.")
except FileNotFoundError as e:
    print(f"Ошибка: {e}. Проверьте правильность пути к ZIP-архиву или имени CSV-файла внутри него.")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

Попытка прочитать первые 2 строк из 'panel.csv' внутри архива '/home/jupyter/datasphere/project/205.zip'...
['"tin"', '"year"', '"reg_number"', '"kind"', '"category"', '"org_name"', '"org_short_name"', '"activity_code_main"', '"region_iso_code"', '"region_code"', '"region"', '"area"', '"settlement"', '"settlement_type"', '"oktmo"', '"lat"', '"lon"', '"start_date"', '"end_date"', '"revenue"', '"expenditure"', '"employees_count"']
['"000000000071"', '2016', '""', '2', '1', '""', '""', '"69.10"', '"RU-TUL"', '71', '"Тульская область"', '""', '"Тула"', '"г"', '"70701000001"', '54.192055', '37.615383', '"2016-08-10"', '"2017-07-10"', '""', '""', '""']


In [7]:
import zipfile
import polars as pl
import os

# Путь, куда будет сохранен извлеченный CSV-файл
extracted_csv_path = '/home/jupyter/datasphere/project/extracted_panel.csv'

print(f"Начинается процесс обработки ZIP-архива: '{zip_file_path}'")
print(f"Будет извлечен CSV-файл: '{csv_file_in_zip_name}' в '{extracted_csv_path}'")
print(f"Затем CSV будет преобразован в Parquet и сохранен в '{parquet_output_path}'")

try:
    # --- Шаг 1: Распаковка CSV из ZIP-архива в отдельный файл ---
    print("\n--- Шаг 1: Распаковка CSV файла из ZIP-архива ---")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        if csv_file_in_zip_name not in zip_ref.namelist():
            raise FileNotFoundError(
                f"Файл '{csv_file_in_zip_name}' не найден внутри архива '{zip_file_path}'. "
                f"Доступные файлы: {zip_ref.namelist()}"
            )
        
        print(f"Извлечение '{csv_file_in_zip_name}' в '{extracted_csv_path}'...")
        zip_ref.extract(csv_file_in_zip_name, path=os.path.dirname(extracted_csv_path))
        
        # Переименовываем, если извлекся не в тот путь
        actual_extracted_path = os.path.join(os.path.dirname(extracted_csv_path), csv_file_in_zip_name)
        if actual_extracted_path != extracted_csv_path:
            os.rename(actual_extracted_path, extracted_csv_path)

    print(f"CSV-файл успешно извлечен и сохранен как: {extracted_csv_path}")

except zipfile.BadZipFile:
    print(f"Ошибка: Файл '{zip_file_path}' не является действительным ZIP-архивом или поврежден.")
except FileNotFoundError as e:
    print(f"Ошибка: {e}. Проверьте правильность пути к ZIP-архиву или имени CSV-файла внутри него.")
except pl.ComputeError as e:
    print(f"Произошла ошибка Polars при обработке файла: {e}")
    print("Возможно, проблемы с определением схемы или форматом данных. Попробуйте настроить параметры `pl.read_csv`.")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

CSV-файл успешно извлечен и сохранен как: /home/jupyter/datasphere/project/extracted_panel.csv


In [8]:
# --- Шаг 2: Чтение CSV-файла и сохранение в Parquet ---
print("\n--- Шаг 2: Чтение CSV-файла и сохранение в Parquet ---")

# Активируем глобальный кэш строк для корректной работы с категориальными данными
with pl.StringCache():
    print(f"Чтение CSV-файла '{extracted_csv_path}' в Polars DataFrame...")
    df = pl.read_csv(
        extracted_csv_path,
        has_header=True,       # Укажите True, если у CSV есть заголовок
        infer_schema_length=10000, # Увеличьте, если Polars плохо определяет типы данных
        separator=';',         # Указываем разделитель точку с запятой
        encoding='utf8',       # Укажите кодировку ('cp1251' или 'latin1', если utf8 не работает)
        dtypes={'tin': pl.Utf8}, # ИНН часто содержит "~~~~~~~~~", поэтому лучше сразу строкой
        null_values=["~~~~~~~~~~~~", ""], # Явно указываем эту строку и пустые строки как NULL
        # ignore_errors=True, # Используйте, если хотите игнорировать строки с ошибками парсинга
    )

print(f"CSV-файл успешно загружен в Polars DataFrame. Количество строк: {len(df)}")
print("Пример заголовков и типов данных:")
print(df.schema)
print("\nПервые 5 строк DataFrame:")
print(df.head())

print(f"\nНачинается запись DataFrame в Parquet файл: '{parquet_output_path}'")
df.write_parquet(
    parquet_output_path,
    compression='snappy' # Рекомендуемое сжатие для Parquet
)

print(f"Файл успешно преобразован в Parquet и сохранен по пути: {parquet_output_path}")



--- Шаг 2: Чтение CSV-файла и сохранение в Parquet ---
Чтение CSV-файла '/home/jupyter/datasphere/project/extracted_panel.csv' в Polars DataFrame...


FileNotFoundError: No such file or directory: /home/jupyter/datasphere/project/extracted_panel.csv

In [2]:
import polars as pl

# --- Настройки ---
# Количество строк, которые вы хотите вывести
n_rows_to_display = 10

print(f"Попытка прочитать первые {n_rows_to_display} строк из CSV-файла: '{csv_file_path}'")

try:
    # Активируем глобальный кэш строк для корректной работы с категориальными данными
    with pl.StringCache():
        # Читаем CSV-файл, ограничивая количество строк для чтения
        # Это предотвратит загрузку всего 15 ГБ в память, если вам нужны только первые строки
        df_sample = pl.read_csv(
            csv_file_path,
            has_header=True,       # Укажите True, если у CSV есть заголовок
            infer_schema_length=10000, # Увеличьте, если Polars плохо определяет типы данных
            separator=';',         # Указываем разделитель точку с запятой
            encoding='utf8',       # Укажите кодировку ('cp1251' или 'latin1', если utf8 не работает)
            dtypes={'tin': pl.Utf8}, # ИНН часто содержит "~~~~~~~~~", поэтому лучше сразу строкой
            null_values=["~~~~~~~~~~~~", ""], # Явно указываем эту строку и пустые строки как NULL
            n_rows=n_rows_to_display, # Ограничиваем чтение только нужным количеством строк
            # ignore_errors=True, # Используйте, если хотите игнорировать строки с ошибками парсинга
        )

    print("\nПервые строки CSV-файла:")
    print(df_sample) # Выводим DataFrame (Polars автоматически выведет первые строки)

    print("\nСхема данных (типы колонок):")
    print(df_sample.schema)

except FileNotFoundError:
    print(f"Ошибка: Файл не найден по пути '{csv_file_path}'. Убедитесь, что CSV был извлечен и путь указан верно.")
except pl.ComputeError as e:
    print(f"Произошла ошибка Polars при обработке файла: {e}")
    print("Возможно, проблемы с определением схемы или форматом данных. Попробуйте настроить параметры `pl.read_csv`.")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

Попытка прочитать первые 10 строк из CSV-файла: '/home/jupyter/datasphere/project/giant_csv_dataset.csv'

Первые строки CSV-файла:
shape: (10, 22)
┌──────────────┬──────┬──────────────┬──────┬───┬────────────┬──────────┬─────────────┬────────────┐
│ tin          ┆ year ┆ reg_number   ┆ kind ┆ … ┆ end_date   ┆ revenue  ┆ expenditure ┆ employees_ │
│ ---          ┆ ---  ┆ ---          ┆ ---  ┆   ┆ ---        ┆ ---      ┆ ---         ┆ count      │
│ str          ┆ i64  ┆ i64          ┆ i64  ┆   ┆ str        ┆ f64      ┆ f64         ┆ ---        │
│              ┆      ┆              ┆      ┆   ┆            ┆          ┆             ┆ i64        │
╞══════════════╪══════╪══════════════╪══════╪═══╪════════════╪══════════╪═════════════╪════════════╡
│ 000000000071 ┆ 2016 ┆ null         ┆ 2    ┆ … ┆ 2017-07-10 ┆ null     ┆ null        ┆ null       │
│ 000000000071 ┆ 2017 ┆ null         ┆ 2    ┆ … ┆ 2017-07-10 ┆ null     ┆ null        ┆ null       │
│ 000000610911 ┆ 2016 ┆ null         ┆ 2    ┆

In [3]:
df_sample

tin,year,reg_number,kind,category,org_name,org_short_name,activity_code_main,region_iso_code,region_code,region,area,settlement,settlement_type,oktmo,lat,lon,start_date,end_date,revenue,expenditure,employees_count
str,i64,i64,i64,i64,str,str,str,str,i64,str,str,str,str,i64,f64,f64,str,str,f64,f64,i64
"""000000000071""",2016,null,2,1,null,null,"""69.10""","""RU-TUL""",71,"""Тульская облас…",null,"""Тула""","""г""",70701000001,54.192055,37.615383,"""2016-08-10""","""2017-07-10""",null,null,null
"""000000000071""",2017,null,2,1,null,null,"""69.10""","""RU-TUL""",71,"""Тульская облас…",null,"""Тула""","""г""",70701000001,54.192055,37.615383,"""2016-08-10""","""2017-07-10""",null,null,null
"""000000610911""",2016,null,2,1,null,null,"""01.11.1""","""RU-ROS""",61,"""Ростовская обл…","""Егорлыкский""","""Шаумяновский""","""х""",60615480101,46.378334,40.564445,"""2016-08-10""","""2017-06-10""",null,null,null
"""000000610911""",2017,null,2,1,null,null,"""01.11.1""","""RU-ROS""",61,"""Ростовская обл…","""Егорлыкский""","""Шаумяновский""","""х""",60615480101,46.378334,40.564445,"""2016-08-10""","""2017-06-10""",null,null,null
"""002461011626""",2016,null,2,1,null,null,"""68.20.2""","""RU-KYA""",24,"""Красноярский к…",null,"""Красноярск""","""г""",4701000001,56.009388,92.85248,"""2016-08-10""","""2017-09-10""",null,null,null
"""002461011626""",2017,null,2,1,null,null,"""68.20.2""","""RU-KYA""",24,"""Красноярский к…",null,"""Красноярск""","""г""",4701000001,56.009388,92.85248,"""2016-08-10""","""2017-09-10""",null,null,null
"""0100000011""",2021,1210100002353,1,1,"""ОБЩЕСТВО С ОГР…","""ООО ""ПСЫГУПС""""","""56.10""","""RU-AD""",1,"""Республика Ады…",null,"""Майкоп""","""г""",79701000001,44.609825,40.100662,"""2021-11-10""","""2024-06-10""",null,null,1
"""0100000011""",2022,1210100002353,1,1,"""ОБЩЕСТВО С ОГР…","""ООО ""ПСЫГУПС""""","""56.10""","""RU-AD""",1,"""Республика Ады…",null,"""Майкоп""","""г""",79701000001,44.609825,40.100662,"""2021-11-10""","""2024-06-10""",1.068e6,1.526e6,0
"""0100000011""",2023,1210100002353,1,1,"""ОБЩЕСТВО С ОГР…","""ООО ""ПСЫГУПС""""","""56.10""","""RU-AD""",1,"""Республика Ады…",null,"""Майкоп""","""г""",79701000001,44.609825,40.100662,"""2021-11-10""","""2024-06-10""",3.5411e7,3.0726e7,12


In [8]:
    
extracted_csv_path = "/home/jupyter/datasphere/project/giant_csv_dataset.csv"
parquet_output_path = "/home/jupyter/datasphere/project/new_205.parquet"


import zipfile
import polars as pl
import os

# --- Шаг 2: Чтение CSV-файла с помощью pl.scan_csv и сохранение в Parquet ---
print("\n--- Шаг 2: Чтение CSV-файла с помощью pl.scan_csv и сохранение в Parquet ---")

# Активируем глобальный кэш строк для корректной работы с категориальными данными
with pl.StringCache():
    print(f"Ленивое чтение CSV-файла '{extracted_csv_path}'...")
    # Используем pl.scan_csv для ленивого чтения, передавая все проверенные параметры
    lazy_df = pl.scan_csv(
        extracted_csv_path,
        has_header=True,       # Укажите True, если у CSV есть заголовок
        infer_schema_length=10000, # Увеличьте, если Polars плохо определяет типы данных
        separator=';',         # Указываем разделитель точку с запятой
        encoding='utf8',       # Укажите кодировку ('cp1251' или 'latin1', если utf8 не работает)
        # --- КЛЮЧЕВОЕ ИЗМЕНЕНИЕ ЗДЕСЬ: Добавляем 'oktmo' как Utf8 ---
        dtypes={'tin': pl.Utf8, 'oktmo': pl.Utf8},
        null_values=["~~~~~~~~~~~~", ""], # Явно указываем эту строку и пустые строки как NULL
        # ignore_errors=True, # Используйте, если хотите игнорировать строки с ошибками парсинга
    )

    print(f"Начинается запись LazyFrame в Parquet файл: '{parquet_output_path}'")
    # Сохраняем LazyFrame в Parquet. Polars оптимизирует этот процесс
    lazy_df.sink_parquet(
        parquet_output_path,
        compression='snappy' # Рекомендуемое сжатие для Parquet
    )

print(f"Файл успешно преобразован в Parquet и сохранен по пути: {parquet_output_path}")
print("\nТеперь вы можете работать с файлом '{}' используя pl.read_parquet() или pl.scan_parquet().".format(parquet_output_path))

# Опционально: Проверка схемы и первых строк из Parquet файла
# df_parquet_check = pl.read_parquet(parquet_output_path, n_rows=5)
# print("\nСхема данных после сохранения в Parquet:")
# print(df_parquet_check.schema)
# print("\nПервые 5 строк из Parquet файла:")
# print(df_parquet_check)



--- Шаг 2: Чтение CSV-файла с помощью pl.scan_csv и сохранение в Parquet ---
Ленивое чтение CSV-файла '/home/jupyter/datasphere/project/giant_csv_dataset.csv'...
Начинается запись LazyFrame в Parquet файл: '/home/jupyter/datasphere/project/new_205.parquet'
Файл успешно преобразован в Parquet и сохранен по пути: /home/jupyter/datasphere/project/new_205.parquet

Теперь вы можете работать с файлом '/home/jupyter/datasphere/project/new_205.parquet' используя pl.read_parquet() или pl.scan_parquet().


In [53]:
import pandas as pd
df = pl.read_parquet(parquet_output_path)
# df = df.to_pandas()
df = df.filter(pl.col("region_code").is_in([77, 78]))
# df = df[df['region'] == 61]
# df.head(10)

In [56]:
df.write_parquet("/home/jupyter/datasphere/project/filtered_205.parquet")

In [20]:
filtered_parquet = "/home/jupyter/datasphere/project/filtered_205.parquet"
df = pl.read_parquet(filtered_parquet).to_pandas()
# df = df.filter(pl.col("revenue").is_not_null())

In [25]:
df.shape
# df = df.filter(pl.col("revenue") != 0.0)
df_not_null = df.dropna(subset =["revenue"])

In [28]:
df_not_null.shape
# df.filter(pl.col("revenue").is_not_null()).filter(pl.col("revenue") != 0.0).filter(pl.col("year") >= 2023)

(3363138, 22)

In [33]:
df_filtered = df[df['activity_code_main'].isin([
    "56.10", "56.10.1", "56.10.2", "56.10.3", "56.30"])]
df_filtered.shape

(249567, 22)

(249567, 22)

In [14]:
# substring = "9AM"
# df = df[df['org_short_name'].notna()]
# mask = df['org_short_name'].str.contains(substring)
# df_filtered = df[mask]
# df_filtered

,tin,year,reg_number,kind,category,org_name,org_short_name,activity_code_main,region_iso_code,region_code,region,area,settlement,settlement_type,oktmo,lat,lon,start_date,end_date,revenue,expenditure,employees_count


In [18]:
df = pd.read_csv("/home/jupyter/datasphere/project/coffee_shops_final.csv")

In [19]:
df

,Unnamed: 0,name,lat,lng,Mon_start,Mon_end,Tue_start,Tue_end,Wed_start,Wed_end,Thu_start,Thu_end,Fri_start,Fri_end,Sat_start,Sat_end,Sun_start,Sun_end,espresso_price,cappuchino_price,specialty_cacao,v60,sells_beans,dog_friendly,with_kitchen,bakery,laptop_friendly,decaf
0,0,milk-beans,55.703506,37.511152,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False
1,1,milk-beans,55.679390,37.468272,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False
2,2,milk-beans,55.754441,37.523975,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False
3,3,milk-beans,55.671443,37.576835,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False
4,4,milk-beans,55.636788,37.724239,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,10:00,22:00,10:00,22:00,150,240,False,True,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,1029,shhegol,59.935057,30.333794,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,110,180,False,True,True,True,False,False,False,False
1030,1030,eklernaya-kler,55.770496,37.633162,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,100,150,False,False,False,True,False,False,False,False
1031,1031,eklernaya-kler,55.772491,37.675998,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,100,150,False,False,False,True,False,False,False,False
1032,1032,erna,55.732911,37.618316,8:00,19:30,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,9:30,21:00,9:30,21:00,130,200,False,True,True,True,False,False,False,False


In [51]:
from typing import Union

def filter_okveds_by_missing_zero_revenue(
    df: pl.DataFrame,
    threshold: float
) -> pl.DataFrame:
    """
    Filters OKVED codes whose proportion of NaN or zero revenue
    does not exceed the given threshold.

    Parameters
    ----------
    df : pl.DataFrame
        Input DataFrame with at least the following columns:
        - activity_code_main: OKVED code (categorical)
        - revenue: numeric column, may contain NaNs or zeros
    threshold : float
        Maximum allowed proportion of NaN or zero revenue
        (between 0 and 1 inclusive).

    Returns
    -------
    pl.DataFrame
        DataFrame of OKVED codes with their null/zero counts,
        total counts, and ratio, filtered by the threshold.
        Columns:
        - activity_code_main
        - null_count
        - zero_count
        - total_count
        - nan_zero_ratio
    """
    # Aggregate counts per OKVED
    agg = (
        df
        .groupby("activity_code_main")
        .agg([
            pl.col("revenue").is_null().sum().alias("null_count"),
            (pl.col("revenue") == 0).sum().alias("zero_count"),
            pl.count().alias("total_count"),
        ])
        .with_columns(
            (
                (pl.col("null_count") + pl.col("zero_count")) 
                / pl.col("total_count")
            ).alias("nan_zero_ratio")
        )
    )

    # Filter by threshold
    result = agg.filter(pl.col("nan_zero_ratio") <= threshold)

    return result

# Sample Polars DataFrame
df = pl.read_parquet("/home/jupyter/datasphere/project/filtered_205.parquet")

# Set threshold (e.g., max 50% NaN+zero)
max_ratio = 0.15

good_okveds = filter_okveds_by_missing_zero_revenue(df, max_ratio)
good_okveds

activity_code_main,null_count,zero_count,total_count,nan_zero_ratio
str,u32,u32,u32,f64
"""51.41.01""",0,0,1,0.0
"""20.14.1""",0,0,3,0.0
"""72.30""",0,0,1,0.0
"""84""",0,0,2,0.0
"""52.32""",0,0,1,0.0
"""24.43.2""",0,0,2,0.0
"""41.12""",0,0,1,0.0
"""05.10.16""",0,0,1,0.0
"""51.52.23""",0,0,1,0.0


In [1]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import psycopg2
import io
import polars as pl
import pandas as pd # <-- Добавляем импорт pandas

# --- КОНФИГУРАЦИЯ ---
DB_HOST = "rc1a-jks7k44l71k5phla.mdb.yandexcloud.net,rc1d-jtk2u5bge5t0pe01.mdb.yandexcloud.net"
DB_PORT = "6432"
DB_NAME = "db1"
DB_USER = "user1"
DB_PASSWORD = "password"

CSV_FILE_PATH = "/home/jupyter/datasphere/project/coffee_shops_final.csv"
TARGET_TABLE_NAME = "test_dataset"

CSV_DELIMITER = ',' # Разделитель в вашем CSV
COLUMNS_FROM_CSV = ['lat', 'lng'] # Колонки, которые нужно считать из CSV

# --- ПОДКЛЮЧЕНИЕ И ЗАГРУЗКА ---

# Устанавливаем соединение с базой данных
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    sslmode='require'
)
conn.autocommit = False # Отключим autocommit, чтобы использовать транзакции
cur = conn.cursor()

# 1. Удаляем таблицу, если она существует, и создаем заново
print(f"Создаем/пересоздаем таблицу {TARGET_TABLE_NAME}...")
cur.execute(f"""
    DROP TABLE IF EXISTS {TARGET_TABLE_NAME};
    CREATE TABLE {TARGET_TABLE_NAME} (
        id BIGSERIAL PRIMARY KEY,
        lat DOUBLE PRECISION,
        lng DOUBLE PRECISION,
        geom GEOMETRY(Point, 4326)
    );
    CREATE INDEX idx_{TARGET_TABLE_NAME}_geom ON {TARGET_TABLE_NAME} USING GIST (geom);
""")

# 2. Создаем временную таблицу для быстрой загрузки данных из CSV
print("Создаем временную таблицу для импорта CSV...")
cur.execute(f"""
    CREATE TEMPORARY TABLE temp_csv_import (
        temp_lat DOUBLE PRECISION,
        temp_lng DOUBLE PRECISION
    );
""")

# 3. Считываем данные из CSV с Polars
print(f"Считываем колонки {COLUMNS_FROM_CSV} из {CSV_FILE_PATH} с помощью Polars...")
df_subset = pl.read_csv(
    CSV_FILE_PATH,
    separator=CSV_DELIMITER,
    has_header=True,
    columns=COLUMNS_FROM_CSV
)

# --- ИСПРАВЛЕНИЕ ОШИБКИ: ИСПОЛЬЗУЕМ PANDAS ДЛЯ ЗАПИСИ В БУФЕР ---
# Конвертируем Polars DataFrame в Pandas DataFrame
pandas_df_subset = df_subset.to_pandas()

# Преобразуем Pandas DataFrame в CSV-строку в буфере памяти
string_buffer = io.StringIO()
pandas_df_subset.to_csv(string_buffer, sep='\t', header=False, index=False) # header=False, index=False очень важны
string_buffer.seek(0) # Переводим указатель в начало буфера

# Преобразуем строковый буфер в байтовый
bytes_buffer = io.BytesIO(string_buffer.getvalue().encode('utf-8'))
# --- КОНЕЦ ИСПРАВЛЕНИЯ ОШИБКИ ---

print("Загружаем данные во временную таблицу temp_csv_import...")
# 5. Быстрая загрузка данных во временную таблицу с использованием COPY
cur.copy_from(bytes_buffer, "temp_csv_import", sep='\t', columns=('temp_lat', 'temp_lng'))

print(f"Вставляем данные из временной таблицы в '{TARGET_TABLE_NAME}' с созданием 'geom'...")
# 6. Вставка данных из временной таблицы в конечную, создавая geom
cur.execute(f"""
    INSERT INTO {TARGET_TABLE_NAME} (lat, lng, geom)
    SELECT
        temp_lat,
        temp_lng,
        ST_SetSRID(ST_MakePoint(temp_lng, temp_lat), 4326)
    FROM
        temp_csv_import;
""")

conn.commit() # Применяем изменения транзакции
cur.close()
conn.close()

print(f"Данные успешно загружены в таблицу '{TARGET_TABLE_NAME}' с колонкой 'geom'.")

Создаем/пересоздаем таблицу test_dataset...
Создаем временную таблицу для импорта CSV...
Считываем колонки ['lat', 'lng'] из /home/jupyter/datasphere/project/coffee_shops_final.csv с помощью Polars...
Загружаем данные во временную таблицу temp_csv_import...
Вставляем данные из временной таблицы в 'test_dataset' с созданием 'geom'...
Данные успешно загружены в таблицу 'test_dataset' с колонкой 'geom'.


In [ ]:
import requests
import pandas as pd

# Отправка запроса
overpass_url = "https://overpass-api.de/api/interpreter"

moscow = "Москва".encode('utf-8')
query = f"""
[out:json][timeout:3600];
area["name"="{moscow}"]->.moscow;
(
  nwr["building"="office"];
  nwr["office"];
  nwr["amenity"="coworking_space"];
);
out center geom;
"""

response = requests.post(overpass_url, data=query)

In [19]:
import requests
import pandas as pd

# Отправка запроса
overpass_url = "https://overpass-api.de/api/interpreter"

bbox_moscow_center = "55.73,37.58,55.77,37.65" # Юг, Запад, Север, Восток

query = f"""
[out:json][timeout:3600];
(
  nwr["building"="office"]({bbox_moscow_center});
  nwr["office"]({bbox_moscow_center});
  nwr["amenity"="coworking_space"]({bbox_moscow_center});
);
out center geom;
"""

response = requests.post(overpass_url, data=query)


In [20]:
# Парсинг результатов

data = response.json()
stations = []
for element in data["elements"]:
    stations.append({
        "id": element["id"],
        "name": element["tags"].get("name", "Unknown"),
        # "lat": element["lat"],
        # "lon": element["lon"]
    })

geo_df = pd.DataFrame(stations)
print(f"Найдено станций: {len(geo_df)}")
print(geo_df.head())
data

Найдено станций: 2322
          id                          name
0  530204987  Центральная акцизная таможня
1  622006644          Посольство Австралии
2  698892110           Консульство Австрии
3  698892149           Посольство Словении
4  698892150             Посольство Алжира


{'version': 0.6,
 'generator': 'Overpass API 0.7.62.7 375dc00a',
 'osm3s': {'timestamp_osm_base': '2025-07-10T12:09:03Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 530204987,
   'lat': 55.7467403,
   'lon': 37.6471724,
   'tags': {'amenity': 'customs',
    'contact:phone': '+7 495 9158127',
    'contact:website': 'http://cat.customs.ru/',
    'name': 'Центральная акцизная таможня',
    'office': 'government'}},
  {'type': 'node',
   'id': 622006644,
   'lat': 55.7525862,
   'lon': 37.6424555,
   'tags': {'addr:city': 'Москва',
    'addr:housenumber': '10',
    'addr:postcode': '109028',
    'addr:street': 'Подколокольный переулок',
    'country': 'AU',
    'diplomatic': 'embassy',
    'embassy': 'yes',
    'name': 'Посольство Австралии',
    'name:ca': "Ambaixada d'Austràlia",
    'name:en': 'Embassy of Australia',
    'name:fr': "Ambassade d'Australie",
    'office

In [18]:
import requests
import pandas as pd


overpass_url = "https://overpass-api.de/api/interpreter"

# moscow = "Москва".encode('utf-8')
# query = f"""
# [out:json][timeout:3600];
# area["name"="{moscow}"]->.moscow;
# (
#   nwr["building"="office"];
#   nwr["office"];
#   nwr["amenity"="coworking_space"];
# );
# out center geom;
# """



def get_poi(city_name):
    city_name = city_name.encode('utf-8')
    query = f"""
    [out:json][timeout:900];
    area["name"="{city_name}"]->.searchArea;
    (
      nwr["building"="office"](area.searchArea);
      nwr["amenity"="coworking_space"](area.searchArea);
    );
    out center;
    """
    response = requests.post("https://overpass-api.de/api/interpreter", data=query)
    return response.json()

def parse_response(data):
    features = []
    for element in data["elements"]:
        # Для узлов (nodes) используем родные координаты
        if element["type"] == "node":
            lat, lon = element["lat"], element["lon"]
        # Для путей (ways) и отношений (relations) используем вычисленный центр
        elif "center" in element:
            lat, lon = element["center"]["lat"], element["center"]["lon"]
        else:
            continue
        
        features.append({
            "id": element["id"],
            "type": element["type"],
            "name": element["tags"].get("name", "Unnamed"),
            "category": "office" if "building" in element["tags"] else "coworking",
            "lat": lat,
            "lon": lon
        })
    return pd.DataFrame(features)

# Пример использования
moscow_data = get_poi("Москва")
spb_data = get_poi("Санкт-Петербург")

df_moscow = parse_response(moscow_data)
df_spb = parse_response(spb_data)

print(f"Найдено в Москве: {len(df_moscow)} объектов")
print(f"Найдено в СПб: {len(df_spb)} объектов")

# Объединение результатов
final_df = pd.concat([df_moscow, df_spb], ignore_index=True)
final_df.to_csv("business_centers.csv", index=False)
moscow_data

Найдено в Москве: 0 объектов
Найдено в СПб: 0 объектов


{'version': 0.6,
 'generator': 'Overpass API 0.7.62.7 375dc00a',
 'osm3s': {'timestamp_osm_base': '2025-07-10T12:09:03Z',
  'timestamp_areas_base': '2025-07-09T01:47:14Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': []}